In [ ]:
import torch
from torch.utils.data import DataLoader
from torch import nn
import torch.optim as optim
import torchvision.models as models

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'functions')))
from dataset import ChestXrayDataset
from train import train
from evaluation import plot_results ,eval_on_metrics
from gradcam import get_heatmap_for_resnet

In [ ]:
IMAGE_PATH = "../archive/"
import glob

# Tüm alt klasörlerdeki jpg ve png dosyalarını alalım
image_paths = glob.glob(IMAGE_PATH + "**/images/*.[jp][pn]g", recursive=True)

print(f"Toplam {len(image_paths)} resim bulundu.")

In [ ]:
TRAIN_PATH = '../data/AP_PA_Train.xlsx'
TEST_PATH = '../data/AP_PA_Test.xlsx'
VAL_PATH = '../data/AP_PA_Validation.xlsx'

In [ ]:
num_classes = 2
EPOCHS = 30

In [ ]:
model = models.resnet50(pretrained=True)

for param in model.parameters():
    param.requires_grad = True

in_features = model.fc.in_features
model.fc = nn.Linear(in_features, num_classes)

for param in model.fc.parameters():
    param.requires_grad = True

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
print(device)

In [ ]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [ ]:
train_dataset = ChestXrayDataset(TRAIN_PATH, image_paths,transform=transform)
val_dataset = ChestXrayDataset(TEST_PATH,image_paths, transform=transform)
test_dataset = ChestXrayDataset(VAL_PATH,image_paths, transform=transform)

In [ ]:
print("Train size : ",len(train_dataset))
print("Validation size : ",len(val_dataset))
print("Test size : ",len(test_dataset))

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True,num_workers=20)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=True,num_workers=20)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True,num_workers=20)

In [ ]:
train_losses, train_accuracies, val_losses, val_accuracies = train(model, train_dataloader, val_dataloader, criterion, optimizer, device,num_epochs=EPOCHS,patience=5)

In [ ]:
print(f"num_epochs: {EPOCHS}")
print(f"train_losses: {len(train_losses)}")
print(f"val_losses: {len(val_losses)}")
print(f"train_accuracies: {len(train_accuracies)}")
print(f"val_accuracies: {len(val_accuracies)}")


In [ ]:
plot_results(train_losses, train_accuracies, val_losses, val_accuracies)

In [ ]:
eval_on_metrics(model, test_dataloader)

In [ ]:
test_dataset[0]

In [ ]:
model = models.resnet50(pretrained=True) 

# Son katmanı 2 sınıflı yap
model.fc = nn.Linear(model.fc.in_features, 2)

checkpoint_path = 'models/best_model.pth'
state_dict = torch.load(checkpoint_path)

# 3. state_dict'i modele yükle
model.load_state_dict(state_dict)

# 4. Modele eval modunu ver (inference için)
model.eval()

In [ ]:
test_image_path = test_dataset.get_path(0)

In [ ]:
get_heatmap_for_resnet(model,test_image_path)